In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Let´s Read The Data And Create A Singel DataFrame

In [76]:
#Define the Sheets Name
sheets = ['Jan23','Mar23','Abri23','Mai23','Jul23','Ago23']

# Create an empty dictionary to store DataFrames
dfs = {}

# For loop to read every sheet and create a dataframe for each one
for sheet in sheets:
    df = pd.read_excel('EDUCABIZ PT - KPI Utilização - Dashboard .xlsx',sheet_name=sheet)
    
    # Store the DataFrame in the dictionary with the sheet name as the key
    dfs[sheet] = df


__Take A Look To each Dataframe__

WE want to have the following columns:
* escola
* slug
* tutores
* second_tutores
* docs_fiscais (15_dias)
* mensagens (7_dias)
* atividades (7_dias)
* relatorios_diarios (7_dias)
* avaliacoes (7_dias)
* menus (7_dias)
* eventos (15_dias)

So let´s clean each column from each dataframe in order to have a dataframe like eith all the previous columns

In [77]:
# Jan23
dfs['Jan23'].head()

,Escola,Slug,Tutores,Second-Tutor,Docs fiscais (-15 dias),Mensagens (-7 dias),Actividades (-30 dias),Rel. Diários (-30 dias),atividades,menus \n(-30 dias),eventos \n(-30 dias)
0,100 + Centro de Estudos de Murça,https://centrodeestudosdemurca.educabiz.com,49,0,31,14,0,334,0,0,0
1,8Tempos-Associação Cultural,https://8tempos.educabiz.com,0,0,94,108,0,0,0,0,0
2,ABLA,https://abla.educabiz.com,411,104,152,1988,14,2098,14,20,1
3,Abrigo Infantil das Laranjeiras - SCMSJM,https://abrigoinfantildaslaranjeiras.educabiz.com,87,0,0,217,78,829,78,0,0
4,Academia CIPE,https://eborlandia.educabiz.com,0,0,144,93,0,0,0,0,0


In [78]:
dfs['Jan23'].rename(columns={'Escola':'escola',
                             'Slug':'slug',
                             'Tutores':'tutores',
                             'Second-Tutor':'second_tutor',
                             'Docs fiscais (-15 dias)': 'docs_fiscais (15_dias)',
                             'Mensagens (-7 dias)':'mensagens (7_dias)',
                             'Actividades (-30 dias)':'atividades (7_dias)',
                             'Rel. Diários (-30 dias)':'relatorios_diarios (7_dias)',
                             'menus \n(-30 dias)':'menus (7_dias)',
                             'eventos \n(-30 dias)':'eventos (15_dias)'},inplace=True)
dfs['Jan23'].head()

,escola,slug,tutores,second_tutor,docs_fiscais (15_dias),mensagens (7_dias),atividades (7_dias),relatorios_diarios (7_dias),atividades,menus (7_dias),eventos (15_dias)
0,100 + Centro de Estudos de Murça,https://centrodeestudosdemurca.educabiz.com,49,0,31,14,0,334,0,0,0
1,8Tempos-Associação Cultural,https://8tempos.educabiz.com,0,0,94,108,0,0,0,0,0
2,ABLA,https://abla.educabiz.com,411,104,152,1988,14,2098,14,20,1
3,Abrigo Infantil das Laranjeiras - SCMSJM,https://abrigoinfantildaslaranjeiras.educabiz.com,87,0,0,217,78,829,78,0,0
4,Academia CIPE,https://eborlandia.educabiz.com,0,0,144,93,0,0,0,0,0


In [79]:
# let´s drop the column 'atividades' and create a new column for 'avaliacoes' assuming that all the 'escolas' has a value 0
dfs['Jan23'].drop(columns=['atividades'],inplace=True)
dfs['Jan23']['avaliacoes (7_dias)'] = 0
dfs['Jan23'] = dfs['Jan23'][['escola', 'slug','tutores', 'second_tutor', 'docs_fiscais (15_dias)','mensagens (7_dias)','atividades (7_dias)','relatorios_diarios (7_dias)','avaliacoes (7_dias)','menus (7_dias)','eventos (15_dias)']]
dfs['Jan23'].head()

,escola,slug,tutores,second_tutor,docs_fiscais (15_dias),mensagens (7_dias),atividades (7_dias),relatorios_diarios (7_dias),avaliacoes (7_dias),menus (7_dias),eventos (15_dias)
0,100 + Centro de Estudos de Murça,https://centrodeestudosdemurca.educabiz.com,49,0,31,14,0,334,0,0,0
1,8Tempos-Associação Cultural,https://8tempos.educabiz.com,0,0,94,108,0,0,0,0,0
2,ABLA,https://abla.educabiz.com,411,104,152,1988,14,2098,0,20,1
3,Abrigo Infantil das Laranjeiras - SCMSJM,https://abrigoinfantildaslaranjeiras.educabiz.com,87,0,0,217,78,829,0,0,0
4,Academia CIPE,https://eborlandia.educabiz.com,0,0,144,93,0,0,0,0,0


In [80]:
# Let´s Cpnvert the Columns atividades, relatorios_diarios, avaliacoes e menus  to 7 days and the column eventos to 15 days

# Function to convert values to 30 days
def convert_7days(value):
    return round((7 * value) / 30,0)

# Function to convert to 15 days
def convert_15days(value):
    return round((15 * value) / 30,0)

# Specifying the DataFrame
df_jan23 = dfs['Jan23']

# Columns to be converted to 7 days
columns_7days = ['atividades (7_dias)', 'relatorios_diarios (7_dias)', 'avaliacoes (7_dias)', 'menus (7_dias)']

# Applying the conversion function to the specified columns
for col in columns_7days:
    df_jan23[col] = df_jan23[col].apply(convert_7days)
    
# Columns to be converted to 15 days
columns_15days = ['eventos (15_dias)']

# Applying the conversion function to the specified columns
for col in columns_15days:
    df_jan23[col] = df_jan23[col].apply(convert_15days)

# Update the 'Jan23' DataFrame within the 'dfs' dictionary
dfs['Jan23'] = df_jan23

dfs['Jan23']

,escola,slug,tutores,second_tutor,docs_fiscais (15_dias),mensagens (7_dias),atividades (7_dias),relatorios_diarios (7_dias),avaliacoes (7_dias),menus (7_dias),eventos (15_dias)
0,100 + Centro de Estudos de Murça,https://centrodeestudosdemurca.educabiz.com,49,0,31,14,0.0,78.0,0.0,0.0,0.0
1,8Tempos-Associação Cultural,https://8tempos.educabiz.com,0,0,94,108,0.0,0.0,0.0,0.0,0.0
2,ABLA,https://abla.educabiz.com,411,104,152,1988,3.0,490.0,0.0,5.0,0.0
3,Abrigo Infantil das Laranjeiras - SCMSJM,https://abrigoinfantildaslaranjeiras.educabiz.com,87,0,0,217,18.0,193.0,0.0,0.0,0.0
4,Academia CIPE,https://eborlandia.educabiz.com,0,0,144,93,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
777,Vírgulas e Vértices,https://virgulasevertices.educabiz.com,17,0,26,0,0.0,0.0,0.0,0.0,0.0
778,WIP - Momentos de Aprendizagem,https://wip.educabiz.com,60,4,2,94,0.0,48.0,0.0,0.0,1.0
779,XokDance,https://xokdance.educabiz.com,13,0,19,2,0.0,0.0,0.0,0.0,2.0
780,YMCA Setúbal,https://ymcasetubal.educabiz.com,282,11,0,1227,0.0,766.0,0.0,0.0,2.0


In [81]:
# Mar23
dfs['Mar23'].head()

,Escola,Slug,Tutores,Second-Tutor,Docs fiscais (-15 dias),Mensagens (-7 dias),Actividades (-30 dias),Rel. Diários (-30 dias),Avaliações,menus \n(-30 dias),eventos \n(-30 dias)
0,CSP São Martinho de Candoso 2,https://cspmcandoso2.educabiz.com,0,0,0,0,0,0,0,0,0
1,Best in Class,https://bestinclass.educabiz.com,0,0,0,0,0,0,0,0,0
2,Projeto Em Flor - Crescer Livre,https://projetoemflor.educabiz.com,12,0,32,174,0,0,0,0,1
3,A Ovelhinha,https://aovelhinha.educabiz.com,0,0,0,0,0,0,0,0,0
4,Colégio Nossa Senhora das Gracas,https://cnossasenhoradasgracas.educabiz.com,0,0,0,0,0,0,0,0,0


In [83]:
dfs['Mar23'].rename(columns={'Escola':'escola',
                             'Slug':'slug',
                             'Tutores':'tutores',
                             'Second-Tutor':'second_tutor',
                             'Docs fiscais (-15 dias)': 'docs_fiscais (15_dias)',
                             'Mensagens (-7 dias)':'mensagens (7_dias)',
                             'Actividades (-30 dias)':'atividades (7_dias)',
                             'Rel. Diários (-30 dias)':'relatorios_diarios (7_dias)',
                             'Avaliações':'avaliacoes (7_dias)',
                             'menus \n(-30 dias)':'menus (7_dias)',
                             'eventos \n(-30 dias)':'eventos (15_dias)'},inplace=True)
dfs['Mar23']

,escola,slug,tutores,second_tutor,docs_fiscais (15_dias),mensagens (7_dias),atividades (7_dias),relatorios_diarios (7_dias),avaliacoes (7_dias),menus (7_dias),eventos (15_dias)
0,CSP São Martinho de Candoso 2,https://cspmcandoso2.educabiz.com,0,0,0,0,0,0,0,0,0
1,Best in Class,https://bestinclass.educabiz.com,0,0,0,0,0,0,0,0,0
2,Projeto Em Flor - Crescer Livre,https://projetoemflor.educabiz.com,12,0,32,174,0,0,0,0,1
3,A Ovelhinha,https://aovelhinha.educabiz.com,0,0,0,0,0,0,0,0,0
4,Colégio Nossa Senhora das Gracas,https://cnossasenhoradasgracas.educabiz.com,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
742,Sonho Audaz,https://sonhoaudaz.educabiz.com,74,46,142,170,86,1082,6181,0,1
743,Colégio Mundo do Colibri,https://mundo-colibri.educabiz.com,141,18,176,1343,0,1875,6932,0,0
744,Zebra,https://zebra.educabiz.com,0,0,100,98,0,0,0,0,0
745,Simplio,https://simplio.educabiz.com,0,0,185,188,0,0,0,0,0


In [84]:
# Let´s Cpnvert the Columns atividades, relatorios_diarios, avaliacoes e menus  to 7 days and the column eventos to 15 days

# Specifying the DataFrame
df_mar23 = dfs['Mar23']

# Columns to be converted to 7 days
columns_7days = ['atividades (7_dias)', 'relatorios_diarios (7_dias)', 'avaliacoes (7_dias)', 'menus (7_dias)']

# Applying the conversion function to the specified columns
for col in columns_7days:
    df_mar23[col] = df_mar23[col].apply(convert_7days)
    
# Columns to be converted to 15 days
columns_15days = ['eventos (15_dias)']

# Applying the conversion function to the specified columns
for col in columns_15days:
    df_mar23[col] = df_mar23[col].apply(convert_15days)

# Update the 'Jan23' DataFrame within the 'dfs' dictionary
dfs['Mar23'] = df_mar23

dfs['Mar23']

,escola,slug,tutores,second_tutor,docs_fiscais (15_dias),mensagens (7_dias),atividades (7_dias),relatorios_diarios (7_dias),avaliacoes (7_dias),menus (7_dias),eventos (15_dias)
0,CSP São Martinho de Candoso 2,https://cspmcandoso2.educabiz.com,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,Best in Class,https://bestinclass.educabiz.com,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,Projeto Em Flor - Crescer Livre,https://projetoemflor.educabiz.com,12,0,32,174,0.0,0.0,0.0,0.0,0.0
3,A Ovelhinha,https://aovelhinha.educabiz.com,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,Colégio Nossa Senhora das Gracas,https://cnossasenhoradasgracas.educabiz.com,0,0,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
742,Sonho Audaz,https://sonhoaudaz.educabiz.com,74,46,142,170,20.0,252.0,1442.0,0.0,0.0
743,Colégio Mundo do Colibri,https://mundo-colibri.educabiz.com,141,18,176,1343,0.0,438.0,1617.0,0.0,0.0
744,Zebra,https://zebra.educabiz.com,0,0,100,98,0.0,0.0,0.0,0.0,0.0
745,Simplio,https://simplio.educabiz.com,0,0,185,188,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Abri23
dfs['Abri23'].head()

,Escola,Slug,Tutores,Second-Tutor,Docs fiscais (-15 dias),Mensagens (-7 dias),Actividades (-7 dias),Rel. Diários (-7 dias),Avaliações\n(-7 dias),menus \n(-7 dias),eventos \n(-15 dias)
0,Traquinauta,https://traquinauta.educabiz.com,0,0,0,0,0,0,0,0,0
1,Karolas - Sala de Estudo,https://karolas.educabiz.com,0,0,0,0,0,0,0,0,0
2,Academia das Chupetas,https://academiadaschupetas.educabiz.com,0,0,0,0,0,0,0,0,0
3,UF Centro Histórico do Porto,https://uf-centrohistoricodoporto.educabiz.com,0,0,0,0,0,0,0,0,0
4,Os Pirralhos do Marquês,https://pirralhos.educabiz.com,0,0,0,0,0,0,0,0,0


In [85]:
dfs['Abri23'].rename(columns={'Escola':'escola',
                             'Slug':'slug',
                             'Tutores':'tutores',
                             'Second-Tutor':'second_tutor',
                             'Docs fiscais (-15 dias)': 'docs_fiscais (15_dias)',
                             'Mensagens (-7 dias)':'mensagens (7_dias)',
                             'Actividades (-7 dias)':'atividades (7_dias)',
                             'Rel. Diários (-7 dias)':'relatorios_diarios (7_dias)',
                             'Avaliações\n(-7 dias)':'avaliacoes (7_dias)',
                             'menus \n(-7 dias)':'menus (7_dias)',
                             'eventos \n(-15 dias)':'eventos (15_dias)'},inplace=True)
dfs['Abri23'].head()

,escola,slug,tutores,second_tutor,docs_fiscais (15_dias),mensagens (7_dias),atividades (7_dias),relatorios_diarios (7_dias),avaliacoes (7_dias),menus (7_dias),eventos (15_dias)
0,Traquinauta,https://traquinauta.educabiz.com,0,0,0,0,0,0,0,0,0
1,Karolas - Sala de Estudo,https://karolas.educabiz.com,0,0,0,0,0,0,0,0,0
2,Academia das Chupetas,https://academiadaschupetas.educabiz.com,0,0,0,0,0,0,0,0,0
3,UF Centro Histórico do Porto,https://uf-centrohistoricodoporto.educabiz.com,0,0,0,0,0,0,0,0,0
4,Os Pirralhos do Marquês,https://pirralhos.educabiz.com,0,0,0,0,0,0,0,0,0


In [ ]:
# Mai23
dfs['Mai23'].head()

,name,?column?,tutores_used,secondtutores_used,documentos,mensagens,atividades,relatorios,progresso,menus,eventos
0,Polegarzinho,https://polegarzinho.educabiz.com,0,0,0,0,0,0,0,0,0
1,Voz Alegre,https://vozalegre.educabiz.com,0,0,0,0,0,0,0,0,0
2,S. C. M. Montijo,https://scmmontijo.educabiz.com,0,0,0,0,0,0,0,0,0
3,Casa do Sagrado Coração de Jesus,https://casascjesus.educabiz.com,0,0,0,0,0,0,0,0,0
4,Santa Casa da Misericórdia de Lousada,https://scmlousada.educabiz.com,0,0,0,0,0,0,0,0,0


In [86]:
dfs['Mai23'].rename(columns={'name':'escola',
                             '?column?':'slug',
                             'tutores_used':'tutores',
                             'secondtutores_used':'second_tutor',
                             'documentos': 'docs_fiscais (15_dias)',
                             'mensagens':'mensagens (7_dias)',
                             'atividades':'atividades (7_dias)',
                             'relatorios':'relatorios_diarios (7_dias)',
                             'progresso':'avaliacoes (7_dias)',
                             'menus':'menus (7_dias)',
                             'eventos':'eventos (15_dias)'},inplace=True)
dfs['Mai23']

,escola,slug,tutores,second_tutor,docs_fiscais (15_dias),mensagens (7_dias),atividades (7_dias),relatorios_diarios (7_dias),avaliacoes (7_dias),menus (7_dias),eventos (15_dias)
0,Polegarzinho,https://polegarzinho.educabiz.com,0,0,0,0,0,0,0,0,0
1,Voz Alegre,https://vozalegre.educabiz.com,0,0,0,0,0,0,0,0,0
2,S. C. M. Montijo,https://scmmontijo.educabiz.com,0,0,0,0,0,0,0,0,0
3,Casa do Sagrado Coração de Jesus,https://casascjesus.educabiz.com,0,0,0,0,0,0,0,0,0
4,Santa Casa da Misericórdia de Lousada,https://scmlousada.educabiz.com,0,0,0,0,0,0,0,0,0


In [ ]:
# Jul23
dfs['Jul23'].head()

,Escola,Slug,Tutores,Second-Tutor,Docs fiscais (-15 dias),Mensagens (-7 dias),Actividades (-7 dias),Rel. Diários (-7 dias),Avaliações\n(-7 dias),menus \n(-7 dias),eventos \n(-15 dias)
0,Centro Social do Divino Espírito Santo,https://centrosdes.educabiz.com,0,0,0,0,0,0,NaN,NaN,NaN
1,Centro de Solidariedade Social do Paião,https://jardimdeinfanciapaiao.educabiz.com,0,0,0,0,0,0,NaN,NaN,NaN
2,Externato O Pintainho,https://opintainho.educabiz.com,0,0,0,0,0,0,NaN,NaN,NaN
3,Centro Social São Cristóvão,https://centrosocialsaocristovao.educabiz.com,0,0,0,0,0,0,NaN,NaN,NaN
4,APPACDM-CEP Fontela,https://cepfontela.educabiz.com,0,0,0,0,0,0,NaN,NaN,NaN


In [87]:
dfs['Jul23'].rename(columns={'Escola':'escola',
                             'Slug':'slug',
                             'Tutores':'tutores',
                             'Second-Tutor':'second_tutor',
                             'Docs fiscais (-15 dias)': 'docs_fiscais (15_dias)',
                             'Mensagens (-7 dias)':'mensagens (7_dias)',
                             'Actividades (-7 dias)':'atividades (7_dias)',
                             'Rel. Diários (-7 dias)':'relatorios_diarios (7_dias)',
                             'Avaliações\n(-7 dias)':'avaliacoes (7_dias)',
                             'menus \n(-7 dias)':'menus (7_dias)',
                             'eventos \n(-15 dias)':'eventos (15_dias)'},inplace=True)
dfs['Jul23']

,escola,slug,tutores,second_tutor,docs_fiscais (15_dias),mensagens (7_dias),atividades (7_dias),relatorios_diarios (7_dias),avaliacoes (7_dias),menus (7_dias),eventos (15_dias)
0,Centro Social do Divino Espírito Santo,https://centrosdes.educabiz.com,0,0,0,0,0,0,NaN,NaN,NaN
1,Centro de Solidariedade Social do Paião,https://jardimdeinfanciapaiao.educabiz.com,0,0,0,0,0,0,NaN,NaN,NaN
2,Externato O Pintainho,https://opintainho.educabiz.com,0,0,0,0,0,0,NaN,NaN,NaN
3,Centro Social São Cristóvão,https://centrosocialsaocristovao.educabiz.com,0,0,0,0,0,0,NaN,NaN,NaN
4,APPACDM-CEP Fontela,https://cepfontela.educabiz.com,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
783,DNA – Dance N’ Arts School,https://dna.educabiz.com,0,0,17,672,0,0,NaN,NaN,NaN
784,Sonho Audaz,https://sonhoaudaz.educabiz.com,79,48,26,130,3,262,NaN,NaN,NaN
785,Colégio Mundo do Colibri,https://mundo-colibri.educabiz.com,142,26,63,1536,0,195,NaN,NaN,NaN
786,Simplio,https://simplio.educabiz.com,0,0,47,180,0,0,NaN,NaN,NaN


In [ ]:
# Ago23
dfs['Ago23'].head()

,name,slug,tutors,secondtutors,docs,msgs,activities,reports,progress,menus,events
0,Creche Olhos de Água,https://scmaolhosdeagua.educabiz.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Infantário do LNEC,https://lnec.educabiz.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CEJI Gondomar,https://cejigondomar.educabiz.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Centro Social Paroquial de Requião,https://csprequiao.educabiz.com,24.0,NaN,NaN,222.0,NaN,NaN,29.0,NaN,2.0
4,CRIBB,https://cribb.educabiz.com,1.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN


In [88]:
dfs['Ago23'].rename(columns={'name':'escola',
                             'tutors':'tutores',
                             'secondtutors':'second_tutor',
                             'docs': 'docs_fiscais (15_dias)',
                             'msgs':'mensagens (7_dias)',
                             'activities':'atividades (7_dias)',
                             'reports':'relatorios_diarios (7_dias)',
                             'progress':'avaliacoes (7_dias)',
                             'menus':'menus (7_dias)',
                             'events':'eventos (15_dias)'},inplace=True)
dfs['Ago23']

,escola,slug,tutores,second_tutor,docs_fiscais (15_dias),mensagens (7_dias),atividades (7_dias),relatorios_diarios (7_dias),avaliacoes (7_dias),menus (7_dias),eventos (15_dias)
0,Creche Olhos de Água,https://scmaolhosdeagua.educabiz.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Infantário do LNEC,https://lnec.educabiz.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CEJI Gondomar,https://cejigondomar.educabiz.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Centro Social Paroquial de Requião,https://csprequiao.educabiz.com,24.0,NaN,NaN,222.0,NaN,NaN,29.0,NaN,2.0
4,CRIBB,https://cribb.educabiz.com,1.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
814,Sonho Audaz,https://sonhoaudaz.educabiz.com,77.0,46.0,38.0,353.0,NaN,340.0,2278.0,NaN,NaN
815,Colégio Mundo do Colibri,https://mundo-colibri.educabiz.com,141.0,25.0,63.0,411.0,NaN,60.0,358.0,NaN,NaN
816,Zebra,https://zebra.educabiz.com,1.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN
817,Simplio,https://simplio.educabiz.com,NaN,NaN,54.0,196.0,NaN,NaN,NaN,NaN,NaN


__Let´s concaconate every dataset__

In [89]:
# Concatenate the two DataFrames vertically (along rows)
# axis=0 means stacking them vertically, axis=1 would stack them horizontally
df = pd.concat([dfs['Jan23'], dfs['Mar23'], dfs['Abri23'],dfs['Mai23'],dfs['Jul23'],dfs['Ago23']], axis=0)

# Reset the index of the concatenated DataFrame
df.reset_index(drop=True, inplace=True)

In [90]:
df

,escola,slug,tutores,second_tutor,docs_fiscais (15_dias),mensagens (7_dias),atividades (7_dias),relatorios_diarios (7_dias),avaliacoes (7_dias),menus (7_dias),eventos (15_dias)
0,100 + Centro de Estudos de Murça,https://centrodeestudosdemurca.educabiz.com,49.0,0.0,31.0,14.0,0.0,78.0,0.0,0.0,0.0
1,8Tempos-Associação Cultural,https://8tempos.educabiz.com,0.0,0.0,94.0,108.0,0.0,0.0,0.0,0.0,0.0
2,ABLA,https://abla.educabiz.com,411.0,104.0,152.0,1988.0,3.0,490.0,0.0,5.0,0.0
3,Abrigo Infantil das Laranjeiras - SCMSJM,https://abrigoinfantildaslaranjeiras.educabiz.com,87.0,0.0,0.0,217.0,18.0,193.0,0.0,0.0,0.0
4,Academia CIPE,https://eborlandia.educabiz.com,0.0,0.0,144.0,93.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4640,Sonho Audaz,https://sonhoaudaz.educabiz.com,77.0,46.0,38.0,353.0,NaN,340.0,2278.0,NaN,NaN
4641,Colégio Mundo do Colibri,https://mundo-colibri.educabiz.com,141.0,25.0,63.0,411.0,NaN,60.0,358.0,NaN,NaN
4642,Zebra,https://zebra.educabiz.com,1.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN
4643,Simplio,https://simplio.educabiz.com,NaN,NaN,54.0,196.0,NaN,NaN,NaN,NaN,NaN


In [92]:
# Export to csv file
df.to_csv('D:/EDUCABIZ/EDUCABIZ.csv',index=False)